In [1]:
from sklearn.naive_bayes import MultinomialNB

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd

In [2]:
train = pd.read_csv('train.csv')
targets = ['EAP', 'HPL', 'MWS']
print(train.shape)
li = -1
ll = -1
for index, sentence in enumerate(train['text'].values):
    if len(sentence) > ll:
        li = index
        ll = len(sentence)
print('Longest sentence, length', ll)
print(train.values[li])

(19579, 3)
Longest sentence, length 4663
['id27184'
 'Diotima approached the fountain seated herself on a mossy mound near it and her disciples placed themselves on the grass near her Without noticing me who sat close under her she continued her discourse addressing as it happened one or other of her listeners but before I attempt to repeat her words I will describe the chief of these whom she appeared to wish principally to impress One was a woman of about years of age in the full enjoyment of the most exquisite beauty her golden hair floated in ringlets on her shoulders her hazle eyes were shaded by heavy lids and her mouth the lips apart seemed to breathe sensibility But she appeared thoughtful unhappy her cheek was pale she seemed as if accustomed to suffer and as if the lessons she now heard were the only words of wisdom to which she had ever listened The youth beside her had a far different aspect his form was emaciated nearly to a shadow his features were handsome but thin worn 

In [3]:
clf = MultinomialNB()
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', clf),
])

x = train['text']
y = train['author']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33)

text_clf.fit(x_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [4]:
from sklearn import metrics

predicted = text_clf.predict(x_test)

print(metrics.classification_report(y_test, predicted, target_names=targets))

             precision    recall  f1-score   support

        EAP       0.72      0.92      0.81      2615
        HPL       0.94      0.65      0.77      1857
        MWS       0.86      0.79      0.82      1990

avg / total       0.83      0.80      0.80      6462



In [ ]:
test = pd.read_csv('test.csv')

predicted_proba = text_clf.predict_proba(test['text'])

output = pd.DataFrame(data={
    'id': test['id'],
    'EAP': predicted_proba[:,0],
    'HPL': predicted_proba[:,1],
    'MWS': predicted_proba[:,2]
})

output = output[['id', 'EAP', 'HPL', 'MWS']]

output.to_csv('submission.csv', index=False)